Lab 10 Exercises
=================
Please see the example code
and companion video for an
introduction to these topics


In [39]:
# install this very alpha library directly from github
# !pip install https://github.com/mcuringa/cartopy/raw/refs/heads/main/dist/maptools-latest.tar.gz -q

# !pip install census us plotly nbformat -q

from maptools import census_vars, ui
import pandas as pd
import geopandas as gpd

import us
from census import Census

# from google.colab import userdata
# api_key = userdata.get('CENSUS_API_KEY')
import os
api_key = os.getenv('CENSUS_API_KEY')

# creaate one census object we will reuse
c = Census(api_key)


In [14]:
field_map = census_vars.get_table("B05006")
field_map

{'B05006_001E': 'total',
 'B05006_002E': 'europe',
 'B05006_003E': 'europe_northern_europe',
 'B05006_004E': 'europe_northern_europe_denmark',
 'B05006_005E': 'europe_northern_europe_ireland',
 'B05006_006E': 'europe_northern_europe_norway',
 'B05006_007E': 'europe_northern_europe_sweden',
 'B05006_008E': 'europe_northern_europe_united_kingdom_inc_crown_dependencies',
 'B05006_009E': 'europe_northern_europe_united_kingdom_inc_crown_dependencies_united_kingdom_excluding_england_and_scotland',
 'B05006_010E': 'europe_northern_europe_united_kingdom_inc_crown_dependencies_england',
 'B05006_011E': 'europe_northern_europe_united_kingdom_inc_crown_dependencies_scotland',
 'B05006_012E': 'europe_northern_europe_other_northern_europe',
 'B05006_013E': 'europe_western_europe',
 'B05006_014E': 'europe_western_europe_austria',
 'B05006_015E': 'europe_western_europe_belgium',
 'B05006_016E': 'europe_western_europe_france',
 'B05006_017E': 'europe_western_europe_germany',
 'B05006_018E': 'europe_we

Foreign-Born Population
=======================
Table **B05006**, _Place of Birth for the Foreign-Born Population in the United States_,
provides details on the birthplace of foreign-born individuals living in the U.S.

You will see that I trimmed the fields to only include Latin American & Caribbean countries,
and only include "named" countries (not "other" or "unknown" categories).
The "universe" for this table is total foreign-born population, so I pushed in
**B05006_001E** to get the total population as well (i.e. every resident).

Note from the [Census Bureau FAQ](https://www.census.gov/topics/population/foreign-born/about/faq.html):

> The foreign-born population is composed of anyone who is 
> not a U.S. citizen at birth. This includes naturalized U.S. 
> citizens, non-citizen U.S. nationals, lawful permanent 
> residents (immigrants), temporary migrants (such as foreign 
> students), humanitarian migrants (such as refugees and 
> asylees), and unauthorized migrants.

In the code below, I create a subset of variables
from table B05006, and then load them from the census.
I create some lists of column names to make it easier
to get the subsets you might need for the exercises.

After loading the census data for every county in NYS, 
I join it with Tiger shapes for counties in NYS in 
order to add county names to the data.

The resulting DataFrame is called `df`.


In [ ]:
# field_names = census_vars.get_table("B05006")

field_names =  {
  'B01003_001E': 'total_pop',
  'B05006_001E': 'total_foreign_born',
# caribe
 'B05006_141E': 'bahamas',
 'B05006_142E': 'barbados',
 'B05006_143E': 'cuba',
 'B05006_144E': 'dominica',
 'B05006_145E': 'dominican_republic',
 'B05006_146E': 'grenada',
 'B05006_147E': 'haiti',
 'B05006_148E': 'jamaica',
 'B05006_149E': 'st_lucia',
 'B05006_150E': 'st_vincent_and_the_grenadines',
 'B05006_151E': 'trinidad_and_tobago',
 'B05006_152E': 'west_indies',
# central america & Mexico
 'B05006_155E': 'belize',
 'B05006_156E': 'costa_rica',
 'B05006_157E': 'el_salvador',
 'B05006_158E': 'guatemala',
 'B05006_159E': 'honduras',
 'B05006_160E': 'mexico',
 'B05006_161E': 'nicaragua',
 'B05006_162E': 'panama',
# south america
 'B05006_165E': 'argentina',
 'B05006_166E': 'bolivia',
 'B05006_167E': 'brazil',
 'B05006_168E': 'chile',
 'B05006_169E': 'colombia',
 'B05006_170E': 'ecuador',
 'B05006_171E': 'guyana',
 'B05006_172E': 'peru',
 'B05006_173E': 'uruguay',
 'B05006_174E': 'venezuela'
}

# some useful variables for getting subsets of cols
caribe = ['bahamas', 'barbados', 'cuba', 
          'dominica', 'dominican_republic', 
          'grenada', 'haiti', 'jamaica', 'st_vincent_and_the_grenadines', 
          'trinidad_and_tobago', 'west_indies']

central = ['mexico', 'belize', 'costa_rica', 'el_salvador',
           'guatemala', 'honduras', 'nicaragua', 'panama']

south = ['argentina', 'bolivia', 'brazil', 'chile', 'colombia',
         'ecuador', 'guyana', 'peru', 'uruguay', 'venezuela']

# get the census data for every county in NYS
fields = list(field_names.keys())
data = c.acs5.get( fields=fields, geo={'for': 'tract:*', 'in': 'county:047,  state:36'}, year=2022)

df = pd.DataFrame(data)
# I also want to rename county, so I will add that to my dict here
field_names["county"] = "countyfp"
df.rename(columns=field_names, inplace=True)
df["statefp"] = df["state"]
df["state"] = df.statefp.apply(census_vars.lookup_state)
df.head()

,total_pop,total_foreign_born,bahamas,barbados,cuba,dominica,dominican_republic,grenada,haiti,jamaica,...,colombia,ecuador,guyana,peru,uruguay,venezuela,state,countyfp,tract,statefp
0,4974.0,820.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NY,047,000100,36
1,1170.0,367.0,0.0,0.0,0.0,0.0,45.0,13.0,0.0,0.0,...,0.0,15.0,0.0,0.0,0.0,41.0,NY,047,000200,36
2,4057.0,494.0,0.0,0.0,2.0,0.0,17.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22.0,66.0,NY,047,000301,36
3,4432.0,911.0,0.0,0.0,0.0,0.0,23.0,10.0,0.0,0.0,...,2.0,0.0,0.0,18.0,0.0,0.0,NY,047,000501,36
4,2774.0,389.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,11.0,3.0,0.0,12.0,0.0,NY,047,000502,36


Problem 1: `for` loops
-----------------------
Load all of the field names from table B05006
using `field_names = census_vars.get_table("B05006")`

Use `for` loops to create 4 new dictionary objects:

1. `americas`
2. `asia`
3. `europe`
4. `africa`
5. `oceania`

These dicts should have the census variable as the key and
a human-readable label as the variable. Use string methods
to make the names more readable. You probably want to use the
replace() method.


In [34]:
field_names = census_vars.get_table("B05006")
field_names

{'B05006_001E': 'total',
 'B05006_002E': 'europe',
 'B05006_003E': 'europe_northern_europe',
 'B05006_004E': 'europe_northern_europe_denmark',
 'B05006_005E': 'europe_northern_europe_ireland',
 'B05006_006E': 'europe_northern_europe_norway',
 'B05006_007E': 'europe_northern_europe_sweden',
 'B05006_008E': 'europe_northern_europe_united_kingdom_inc_crown_dependencies',
 'B05006_009E': 'europe_northern_europe_united_kingdom_inc_crown_dependencies_united_kingdom_excluding_england_and_scotland',
 'B05006_010E': 'europe_northern_europe_united_kingdom_inc_crown_dependencies_england',
 'B05006_011E': 'europe_northern_europe_united_kingdom_inc_crown_dependencies_scotland',
 'B05006_012E': 'europe_northern_europe_other_northern_europe',
 'B05006_013E': 'europe_western_europe',
 'B05006_014E': 'europe_western_europe_austria',
 'B05006_015E': 'europe_western_europe_belgium',
 'B05006_016E': 'europe_western_europe_france',
 'B05006_017E': 'europe_western_europe_germany',
 'B05006_018E': 'europe_we

Problem 2: sum()
-----------------
Use the `sum()` method to get the total population
of foreign born residents in country of origin.
Create a DataFrame with the results to display the
table in **long** format with 2 columns:
Country, No. Residents.

Use a style formatter to format the table.


In [18]:
table = df[caribe + central + south].sum()
table = table.to_frame().reset_index()
table.columns = ['Country', 'No. Residents']
table = table.sort_values(by='No. Residents', ascending=False)
formats = {
    'Country': lambda x: x.replace('_', ' ').title(),
    'No. Residents': '{:,.0f}'
}
table = table.style.format(formats)
table.set_caption('<h3>Foreign Born Residents in Brooklyn, 2022</h3>(Caribbean and Latin America)')



,Country,No. Residents
7,Jamaica,"64,156"
4,Dominican Republic,"54,589"
6,Haiti,"50,842"
25,Guyana,"43,619"
11,Mexico,"42,460"
9,Trinidad And Tobago,"42,089"
24,Ecuador,"21,351"
5,Grenada,"17,191"
1,Barbados,"15,101"
8,St Vincent And The Grenadines,"10,301"


Problem 3: Merge Census Tracts
-------------------------------
Merge the census data with the Tiger shapefile for
the census tracts in our data set.

Go to <https://www2.census.gov/geo/tiger/TIGER2022/TRACT/>
to find the right file for NYS tracts. Do an
inner join so that you get only the trats we need.

**Notes:**

- either rename some columns or use left_on, right_on arguments
- you will need more than one column to merge on, "TRACTCE" can be repeated in different counties

When you're done, call `explore()` on your new DataFrame
to make sure it's a GeoDataFrame and that it only contains
Brooklyn, NY.


In [33]:
# write your code here
tracts = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2022/TRACT/tl_2022_36_tract.zip")
tracts.rename(columns={'TRACTCE':'tract', "COUNTYFP":"countyfp"}, inplace=True)
origin = tracts[["tract", "countyfp", "geometry"]].merge(df, on=["tract", "countyfp"], how='inner')
origin.explore()

Problem 4: Layered Map
============================
Make a new map with 3 layers:
- % Caribbean population
- % Central American population
- % South American population

Use a different sequential color maps for each layer.
You will have to calculate the percentages to use before
making the map.

In [6]:
# write your code here

Bonus Problem: categorical map
===============================
Use the `idmax()` method to get the country with the
most residents in each tract. Use a categorical color
map with a legend to display this data.


In [46]:
# bonus code
origin["max_origin"] = origin[caribe + central + south].apply(lambda x: x.idxmax(), axis=1)
base = ui.base_map(origin, zoom=12)
origin.explore(m=base, column="max_origin", style_kwds={"fillOpacity":1}, tooltip="max_origin", cmap="tab20", legend=True)